* 方法参考自 The Barra China Equity Model (CNE5)'s 文档

* 请在环境变量中设置`DB_URI`指向数据库

In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
from alphamind.api import *
from PyFin.api import *
from alphamind.analysis.crosssetctions import cross_section_analysis

plt.style.use('ggplot')

In [2]:
"""
Back test parameter settings
"""

start_date = '2016-01-01'
end_date = '2018-02-28'
category = 'sw'
level = 1
freq = '20b'
universe = Universe('ashare_ex')

horizon = map_freq(freq)
ref_dates = makeSchedule(start_date, end_date, freq, 'china.sse')

In [3]:
def risk_factor_analysis(factor_name):
    data_source = os.environ['DB_URI']
    engine = SqlEngine(data_source)
    risk_names = list(set(risk_styles).difference({factor_name}))
    industry_names = list(set(industry_styles).difference({factor_name}))
    constraint_risk = risk_names + industry_names
    
    df = pd.DataFrame(columns=['ret', 'ic', 't.'], dtype=float)

    for ref_date in ref_dates:
        df.loc[ref_date, :] = cross_section_analysis(ref_date,
                                                     factor_name,
                                                     universe,
                                                     horizon,
                                                     constraint_risk,
                                                     engine=engine)
    df.index = pd.to_datetime(df.index)
    return df

In [4]:
candidates_factors = ['BETA', 'SIZE']

In [5]:
%%time
res = [risk_factor_analysis(factor) for factor in candidates_factors]

2019-02-10 00:36:31,655 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:36:31,662 - ALPHA_MIND - INFO - 2016-01-04 00:00:00 is finished with 2772 stocks for BETA
2019-02-10 00:36:31,665 - ALPHA_MIND - INFO - 2016-01-04 00:00:00 risk_exposure: 8.780185359837276e-32
2019-02-10 00:36:34,375 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:36:34,378 - ALPHA_MIND - INFO - 2016-02-01 00:00:00 is finished with 2773 stocks for BETA
2019-02-10 00:36:34,380 - ALPHA_MIND - INFO - 2016-02-01 00:00:00 risk_exposure: 4.033049288078771e-31
2019-02-10 00:36:37,086 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:36:37,088 - ALPHA_MIND - INFO - 2016-03-07 00:00:00 is finished with 2774 stocks for BETA
2019-02-10 00:36:37,091 - ALPHA_MIND - INFO - 2016-03-07 00:00:00 risk_exposure: 2.032537819277293e-31
2019-02-10 00:36:39,868 - ALPHA_MIND - WARNING

2019-02-10 00:38:11,308 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:38:11,311 - ALPHA_MIND - INFO - 2018-02-22 00:00:00 is finished with 3433 stocks for BETA
2019-02-10 00:38:11,313 - ALPHA_MIND - INFO - 2018-02-22 00:00:00 risk_exposure: 3.792625694052645e-06
2019-02-10 00:38:20,225 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:38:20,228 - ALPHA_MIND - INFO - 2016-01-04 00:00:00 is finished with 2772 stocks for SIZE
2019-02-10 00:38:20,230 - ALPHA_MIND - INFO - 2016-01-04 00:00:00 risk_exposure: 2.4238582429739416e-31
2019-02-10 00:38:27,864 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:38:27,866 - ALPHA_MIND - INFO - 2016-02-01 00:00:00 is finished with 2773 stocks for SIZE
2019-02-10 00:38:27,869 - ALPHA_MIND - INFO - 2016-02-01 00:00:00 risk_exposure: 5.674415515478732e-31
2019-02-10 00:38:36,504 - ALPHA_MIND - WARNIN

2019-02-10 00:39:55,211 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:39:55,214 - ALPHA_MIND - INFO - 2018-01-18 00:00:00 is finished with 3386 stocks for SIZE
2019-02-10 00:39:55,218 - ALPHA_MIND - INFO - 2018-01-18 00:00:00 risk_exposure: 2.2368285841177516e-31
2019-02-10 00:39:58,672 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2019-02-10 00:39:58,674 - ALPHA_MIND - INFO - 2018-02-22 00:00:00 is finished with 3433 stocks for SIZE
2019-02-10 00:39:58,677 - ALPHA_MIND - INFO - 2018-02-22 00:00:00 risk_exposure: 4.486309577858139e-07


Wall time: 3min 30s


In [6]:
df = pd.DataFrame()

for f_name, data in zip(candidates_factors, res):
    data['factor'] = f_name
    df = df.append(data)

In [7]:
df['abs t.'] = np.abs(df['t.'])
df[['factor', 'abs t.']].groupby('factor').mean().sort_values('abs t.', ascending=False).head()

,abs t.
factor,
SIZE,4.764919
BETA,2.538497
